Notebook para el desarrollo del proyecto datascience desafio latam

In [1]:
# Procesamiento de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lazypredict


# Machine Learning
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

from lazypredict.Supervised import LazyClassifier
from lazypredict.Supervised import LazyRegressor


# Tratamiento de warnings
import warnings
warnings.simplefilter('ignore')

In [2]:
df_eda = pd.read_csv('Apoyo Tema Propuesto - Automotora/true_car_listings.csv')

In [3]:
df_train = pd.read_csv('Apoyo Tema Propuesto - Automotora/true_cars_train.csv',sep=';' )

In [4]:
df_test = pd.read_csv('Apoyo Tema Propuesto - Automotora/true_cars_test.csv',sep=';' )